In this project, we will start by defining the paths correctly.

In [1]:
import pandas as pd
import os

# DEFINE PATHS
RAW_PATH = os.path.join("..", "data", "raw")
PROCESSED_PATH = os.path.join("..", "data", "processed")

# Check if paths exist to avoid errors
if not os.path.exists(RAW_PATH):
    print(f"WARNING: Data path not found at {RAW_PATH}")
    print("Please ensure your CSV files are in the 'data/raw' folder!")
else:
    print(f"Data path confirmed: {RAW_PATH}")

Data path confirmed: ..\data\raw


In this section, we are loading the 3 key tables and filter out any orders that were canceled or unavailable so we don't train bad data

In [2]:
# 1. Load the Core Tables
print("Loading datasets...")
try:
    orders = pd.read_csv(os.path.join(RAW_PATH, "olist_orders_dataset.csv"))
    items = pd.read_csv(os.path.join(RAW_PATH, "olist_order_items_dataset.csv"))
    reviews = pd.read_csv(os.path.join(RAW_PATH, "olist_order_reviews_dataset.csv"))
    print("Datasets loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading files: {e}")

# 2. Pre-processing Dates
# Convert the string date to a real datetime object
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

# 3. Filter for Valid Demand (Business Logic)
# We only want to train our model on completed sales.
filtered_orders = orders[orders['order_status'] == 'delivered'].copy()

print(f"Orders Loaded: {len(orders)}")
print(f"Orders after filtering for 'Delivered': {len(filtered_orders)}")

Loading datasets...
Datasets loaded successfully.
Orders Loaded: 99441
Orders after filtering for 'Delivered': 96478


We now want to merge the tables together to create our master dataset and save it as a parquet file.

In [3]:
# 4. The "SQL-Style" Joins

# Merge A: Orders + Items
# Inner Join: We only want orders that actually contain items.
merged_df = pd.merge(filtered_orders, items, on="order_id", how="inner")

# Merge B: + Reviews
# Left Join: Keep the sale even if it has no review.
final_df = pd.merge(merged_df, reviews, on="order_id", how="left")

# 5. Clean up columns
# Select only what we need for the LSTM and Prophet models
output_df = final_df[[
    'order_id', 
    'order_purchase_timestamp', 
    'price',                # Sales Amount
    'product_id', 
    'review_comment_message', # Text for NLP
    'review_score'          # Rating (1-5)
]]

# 6. Save to Parquet because it is Faster than CSV
os.makedirs(PROCESSED_PATH, exist_ok=True)
save_path = os.path.join(PROCESSED_PATH, "merged_transactions.parquet")
output_df.to_parquet(save_path, index=False)

print(f"Success! Merged dataset with {len(output_df)} rows saved to {save_path}")
output_df.head()

Success! Merged dataset with 110840 rows saved to ..\data\processed\merged_transactions.parquet


,order_id,order_purchase_timestamp,price,product_id,review_comment_message,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,29.99,87285b34884572647811a353c7ac498a,"Não testei o produto ainda, mas ele veio corre...",4.0
1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,118.70,595fac2a385ac33a80bd5114aec74eb8,Muito bom o produto.,4.0
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,159.90,aa4383b373c6aca5d8797843e5594415,NaN,5.0
3,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,45.00,d0b61bfb1de832b15ba9d266ca96e5b0,O produto foi exatamente o que eu esperava e e...,5.0
4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,19.90,65266b2da20d04dbe00c5c2d3bb7859e,NaN,5.0


In the next step, we want to convert the review texts into a number that our neural network can understand. Since the text is in portuguese, we will use a multilingual Bert Model.

PHASE 2: FEATURE ENGINEERING/SENTIMENT ANALYSIS.
We will start by installing deep learning libraries.

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ------- -------------------------------- 1.8/10.1 MB 8.8 MB/s eta 0:00:01
   -------------- ------------------------- 3.7/10.1 MB 8.8 MB/s eta 0:00:01
   -------------------- ------------------- 5.2/10.1 MB 8.4 MB/s eta 0:00:01
   --------------------------- ------------ 7.1/10.1 MB 8.5 MB/s eta 0:00:01
   ---------------------------------- ----- 8.7/10.1 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------  10.0/10.1 MB 7.9 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 7.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/536.6 kB ? eta -:--:--
   ---------------------------------------- 536.6/536.6 kB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   --------------------- ------------------ 1.6/2.9 MB 8.7 MB/s eta

  DEPRECATION: Building 'torc' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'torc'. Discussion can be found at https://github.com/pypa/pip/issues/6334
